In [1]:
%reset -f
import numpy as np
from scipy.stats import geom
import sys
from datetime import datetime
import os

In [ ]:
output_log_file = "experiment_4_H.log"
log_file = open(output_log_file, 'a')
sys.stdout = log_file
print(output_log_file)
print(datetime.now().strftime("%a %b %d %H:%M:%S %Y"))

experiment_4_H.log
Sun Dec 28 12:13:40 2025
Defense rate = 1, Attack rate = 0
Target c(12,13,14,16) hit probability: 0.135385

Defense rate = 2, Attack rate = 0
Target c(12,13,14,16) hit probability: 0.190049

Defense rate = 3, Attack rate = 0
Target c(12,13,14,16) hit probability: 0.219995




In [3]:
DEFAULT_WEIGHT_VALUE = 0  
image_mode = False
debug_mode = False

%run attack_graph_MIR100.ipynb

In [4]:
defense_rate_list = [1, 2, 3]

In [5]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    
    # Part01: extract hardness values from all edges & append them
    hardness = []
    for i in range(len(route) - 1):
        start_node = route[i]
        end_node = route[i + 1]
        
        # Initialize vars for max weight loop
        weights = []
        # Collect weights for max
        for edge in graph[start_node][end_node].values():
            weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get maximum weight
        max_weight = max(weights) if weights else DEFAULT_WEIGHT_VALUE
        
        min_weights = []
        # Collect weights for min
        for edge in graph[start_node][end_node].values():
            min_weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get minimum weight
        min_weight = min(min_weights) if min_weights else DEFAULT_WEIGHT_VALUE
        
        hardness.append(np.exp(-min_weight))
       
    hardness = np.array(hardness)


    # print(f'Hardness: {hardness}')
    
    # finding attack_rate with GM 
    if attack_rate is None or attack_rate == 0: #if not provided
        
        def geom_mean(x):
            # filter out 0s avoiding log(0)
            positive_values = x[x > 0]
            if len(positive_values) == 0:
                return 1  # if all vals = zero 
            return np.exp(np.mean(np.log(positive_values)))
        
        attack_rate = 1 / geom_mean(hardness)
    
    # Calculate prob dist using geometric dist
    prob = attack_rate / (attack_rate + defense_rate)
    
    x_values = np.arange(len(route))
    
    # Calculate probs mass function 
    pdf_d = prob * (1-prob)**x_values
    
    # Normalize 
    nsum = np.sum(pdf_d)
    ret = pdf_d / nsum
    return ret

In [6]:
%run heuristic_defense.ipynb

In [7]:
sys.stdout = sys.__stdout__
log_file.close()
with open(output_log_file, 'r') as f:
    print(f.read())